# How to generate features in images

**What are features?**

Generating features in image processing context means applying a bunch of filters that describes the data (images) for example, features that describe the regions of interest, edge detetcion and other filters that can enhance the information. Some general features of an Image are :

- Pixel (Brightness) Values or intesity
- Edges
- Textures
- Orientation
- Local Contrast

**Feature Vector :** List of pixel values at a given pixel there and each value corresponds to the pixel value for the feature or the filter that we have actually applied.

- If we have applied 10 filters then the size of feature vector will be 11 Dimensional (Original + Filters).

- If we know what filter is working well with our image then we don't need ML. But if we don't know what combination of these features will describr the region of interest for example, the grey level is same but the texture is different. So we will be needing feature that probably describes the texture of this region.

In [5]:
#A few generic filters...

import cv2
from skimage.filters.rank import entropy
from skimage.morphology import disk
import pandas as pd

img = cv2.imread('images/scratch.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

If you opne the image you can observe that this is tough image to segment using traditional histogram segmentation. As human we can see that the side areas have lots of texture and the scratch i.e., the area in the middle is flat. But how do we segment these regions ?

A good filter to apply in such a scenario would be **Entropy Filter**. Entropy is lack of order or we can say it is a measure of diorder. In context of this image we can say that there is more disorder in the two sides of this image than the flat region in the centre.

In [7]:
from skimage.filters.rank import entropy
from skimage.morphology import disk  #we need to specify `disk` in entropy, it's like specifying a kernel

entropy_img = entropy(img, disk(1))

cv2.imshow('original image', img)
cv2.imshow('entropy', entropy_img)
cv2.waitKey()
cv2.destroyAllWindows()

The region where there is some order (centre) the entropy value is low. So entropy is a good filter to describe this image. Or we can say it extracts the features well from the image. Let's see another example.

In [9]:
import cv2
from skimage.filters.rank import entropy
from skimage.morphology import disk
import pandas as pd

img = cv2.imread('images/Yeast_Cells.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

entropy_img = entropy(img, disk(1))

cv2.imshow('original image', img)
cv2.imshow('entropy', entropy_img)
cv2.waitKey()
cv2.destroyAllWindows()

So in this case `entropy` is definitely not a good filter to describe this image.

Lets try another filter called **Gaussian Blur**

In [10]:
import cv2
from scipy import ndimage as nd

img = cv2.imread('images/Yeast_Cells.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gaussian_img = nd.gaussian_filter(img, sigma=3)

cv2.imshow('original image', img)
cv2.imshow('Gaussian', gaussian_img)
cv2.waitKey()
cv2.destroyAllWindows()

No sure, whether it is desribing anything or not in the image. Hence it may or may not be useful for the machine.

Let's try one more filter called **Sobel Filter** which performs very good in extracting edges

In [11]:
import cv2
from skimage.filters import sobel

img = cv2.imread('images/Yeast_Cells.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sobel_img = sobel(img)

cv2.imshow('original image', img)
cv2.imshow('Sobel', sobel_img)
cv2.waitKey()
cv2.destroyAllWindows()

So sobel image gives us the region with hard edges so this can be a very good filter  to separate the cells in this image.

---

There are n number of filters available, we can find them in the documentation of skimage. But the question now is how to put this information together to create a panda's data frame which we can use as a input in out machine learning algorithms.

In [14]:
import cv2
from skimage.filters.rank import entropy
from skimage.morphology import disk
import pandas as pd

img = cv2.imread('images/scratch.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# img = cv2.imread('imagees/Yeast_Cells.png')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img2 = img.reshape(-1) #to covert the image into one single list i.e., flattening

df = pd.DataFrame()    #creating an empty data frame
df['Original Image'] = img2    #adding a column 'Original Image'and fill it with the list of pixel values of original image


entropy_img = entropy(img, disk(1))    #applying entropy filter

entropy1 = entropy_img.reshape(-1)    #flattening the image obtained after entropy filter
df['Entropy'] = entropy1    #another column for entropy and filling it with the list of pixel values after applying entropy filter
#print(df)

from scipy import ndimage as nd
#import numpy as np
gaussian_img = nd.gaussian_filter(img, sigma=3)
gaussian_img1 = gaussian_img.reshape(-1)
df['Gaussian'] = gaussian_img1


from skimage.filters import sobel
sobel_img = sobel(img)
sobel1 = sobel_img.reshape(-1)
df['Sobel'] = sobel1


print(df)

        Original Image   Entropy  Gaussian  Sobel
0                  105  1.584963       114    0.0
1                  118  2.000000       115    0.0
2                  129  1.500000       116    0.0
3                  129  1.500000       117    0.0
4                  126  1.500000       118    0.0
...                ...       ...       ...    ...
115595             123  2.000000       123    0.0
115596             122  1.500000       123    0.0
115597             123  2.000000       124    0.0
115598             119  2.000000       125    0.0
115599             126  1.584963       125    0.0

[115600 rows x 4 columns]


Now our data frame is ready to go into a machine learning algorithm, where it can train and create a model.